In [5]:
import requests
star_url = 'https://www.starbucks.co.kr/store/getStore.do?r=PQ5XCBUXK5'
payload ={"in_biz_cds": "0","in_scodes": "0","ins_lat": "37.4923734","ins_lng": "127.0272751","search_text": "","p_sido_cd": "01","p_gugun_cd": "","isError": "true","in_distance": "0","in_biz_cd": "","iend": "1000","searchType": "C","set_date": "","rndCod": "946OSHHM6N","all_store": "0","T03": "0","T01": "0","T27": "0","T12": "0","T09": "0","T30": "0","T05": "0","T22": "0","T21": "0","T10": "0","T36": "0","T43": "0","T48": "0","Z9999": "0","P02": "0","P10": "0","P50": "0","P20": "0","P60": "0","P30": "0","P70": "0","P40": "0","P80": "0","whcroad_yn": "0","P90": "0","P01": "0","new_bool": "0"}

r= requests.post(star_url,data=payload).json()


In [27]:
import pandas as pd
df = pd.DataFrame(r['list'])

In [10]:
len(df.addr.str.find('서울') > 1)

612

In [28]:
#lot : 경도 , lat : 위도
a= ['lat','lot','addr']
star_df = df[['addr', 's_name', 'tel', 'fax', 'gugun_name', 'defaultimage', 'lat', 'lot']].copy()


In [29]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 KB 959.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 KB 883.4 kB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


25

In [102]:
import folium
fmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
star_df.gugun_name.unique().size

import json
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()
data = star_df.gugun_name.value_counts()
df = pd.DataFrame(data)
df.reset_index(inplace=True)
df.rename(columns={'count' : 'value'}, inplace=True)
folium.Choropleth(geo_str, 
                  data = df, 
                  columns=['gugun_name', 'value'],
                  fill_color= "PuRd",
                  key_on = 'feature.id').add_to(fmap)
for idx, row in star_df[['s_name','lat','lot']].iterrows():
    folium.Marker(row[['lat','lot']],popup=row.s_name).add_to(fmap)


/home/david/.local/lib/python3.10/site-packages/folium/utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/home/david/.local/lib/python3.10/site-packages/folium/utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/home/david/.local/lib/python3.10/site-packages/folium/utilities.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (

In [99]:
fmap.save('./starbucks_location.html')

In [90]:
fmap

In [88]:
star_df[['lat','lot']].values

array([['37.501087', '127.043069'],
       ['37.510178', '127.022223'],
       ['37.5139309', '127.0206057'],
       ...,
       ['37.615368', '127.076633'],
       ['37.6066536267232', '127.106359790053'],
       ['37.60170912407773', '127.07841136432036']], dtype=object)

In [79]:
pd.DataFrame([[1,2,3],[2,3,4]]).loc[0:,1:]

,1,2
0,2,3
1,3,4


In [116]:
import pandas as pd
import os

csv_file= [x for x in os.walk('./subway/2016/')][0][2:][0]

In [119]:
first = './subway/2016/' + csv_file[0]

DATA FRAME 을 다음과 같이 정의하고자 한다.
날짜,호선,역명,구분,할인,총인원

In [233]:
first_df = pd.read_csv(first, encoding='euc-kr')
tmp_df= first_df[['날짜','호선','역명','구 분','할인']].copy()
tmp = []
for idx, row in first_df.iterrows():
    tmp.append(row.values[5:].sum())
tmp_df['인원'] = tmp
first_df = tmp_df.copy()
total_df = first_df.copy()

In [255]:
from tqdm import tqdm
for x in tqdm(range(11,12)):
    t_df = pd.read_csv('./subway/2016/'+csv_file[x], encoding='euc-kr')
    tmp_df= t_df[['날짜','호선','역명','구 분','할인']].copy()
    tmp = []
    for idx, row in t_df.iterrows():
        tmp.append(row.values[5:].sum())
    tmp_df['인원'] = tmp
    total_df = pd.concat([total_df,tmp_df], axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: "['호선', '구 분', '할인'] not in index"

In [253]:
t_df = pd.read_csv('./subway/2016/'+csv_file[11], encoding='euc-kr')
#역명 날짜 구분
tmp_df= t_df[['날짜','역명','구분']].copy()
tmp_df.rename(columns={'구분':'구 분'},inplace=True)
tmp = []
for idx, row in t_df.iterrows():
    tmp.append(row[3:].sum())
tmp_df['인원'] = tmp
tmp_df
total_df = pd.concat([total_df,tmp_df], axis=0)


In [398]:
subway_df= total_df.groupby(['역명','구 분']).인원.sum().reset_index().copy()

In [366]:
subway_df= total_df.groupby(['역명','구 분']).인원.sum().reset_index().copy()
subway_station = subway_df.역명.unique().tolist()

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='seoul_subway_locator')
geo_station = {}
no_find = []
for station in tqdm(subway_station):
    location = geolocator.geocode(station + ', 서울')
    if location:
        geo_station[station] = [location.latitude, location.longitude]
    else:
        no_find.append(station)
    

  0%|          | 0/119 [00:00<?, ?it/s]

100%|██████████| 119/119 [01:05<00:00,  1.81it/s]


In [367]:
dd = {}
for s,ss in zip(list(map(lambda x: x[:x.find('(')],no_find)), no_find):
    if s == '지축':
        print('x')
        continue
    dd[ss] = s
    

x


In [368]:
for s in no_find:
    if s == '지축(309)':
        print('x')
        continue
    location = geolocator.geocode(dd[s] + ', 서울')
    geo_station[s] = [location.latitude,location.longitude]

x


In [369]:
geo_station['지축(309)'] = [37.6481,126.9137]

In [399]:
subway_df['위도'] = subway_df.역명.apply(lambda x: geo_station[x][0])
subway_df['경도'] = subway_df.역명.apply(lambda x: geo_station[x][1])

In [418]:
subway_df[['위도','경도','인원']].values.tolist()

[[37.4925324, 127.1186382, 3386795.0],
 [37.4925324, 127.1186382, 3206055.0],
 [37.474135849999996, 127.09384934901931, 36264203.0],
 [37.474135849999996, 127.09384934901931, 36727286.0],
 [37.5351109, 127.0947415, 18308815.0],
 [37.5351109, 127.0947415, 18215056.0],
 [37.5400566, 127.0702592, 16695667.0],
 [37.5400566, 127.0702592, 18254700.0],
 [37.57975395, 126.97668069886859, 9400796.0],
 [37.57975395, 126.97668069886859, 9842682.0],
 [37.495401, 127.1239038, 2756047.0],
 [37.495401, 127.1239038, 2750032.0],
 [37.5119797, 127.0017181, 20905993.0],
 [37.5119797, 127.0017181, 21681792.0],
 [37.4939376, 127.0145997, 13747435.0],
 [37.4939376, 127.0145997, 15250765.0],
 [37.4936592, 127.0137025, 5130231.0],
 [37.4936592, 127.0137025, 3775785.0],
 [37.48417755, 126.89575893688135, 23105147.0],
 [37.48417755, 126.89575893688135, 23186215.0],
 [37.5368976, 127.085103, 8867002.0],
 [37.5368976, 127.085103, 8651949.0],
 [37.6370259, 126.91910713728154, 6580907.0],
 [37.6370259, 126.91910713

In [440]:
from datetime import datetime, date, timedelta
import folium
# 서울시 맵
import json

import folium.plugins
fmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")

folium.plugins.HeatMap(subway_df[['위도','경도','인원']].values.tolist()).add_to(fmap)
#승차 Heatmap
up_fmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
folium.plugins.HeatMap(subway_df[subway_df['구 분'] == '승차'][['위도','경도','인원']].values.tolist()).add_to(up_fmap)
#
down_fmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
folium.plugins.HeatMap(subway_df[subway_df['구 분'] == '하차'][['위도','경도','인원']].values.tolist()).add_to(down_fmap)

In [443]:
fmap

In [441]:
up_fmap

In [442]:
down_fmap

In [446]:
subway_df.iloc[0].values

array(['가락시장(340)', '승차', 3386795, 37.4925324, 127.1186382], dtype=object)

In [447]:
#mysql
import pymysql
host='192.168.90.103'
user='david'
password='392766'
db='encore'
charset='utf8'
con =pymysql.connect(host=host,user=user,password=password,db=db,charset=charset)
cur = con.cursor()


In [463]:
subway_df

,역명,구 분,인원,위도,경도
0,가락시장(340),승차,3386795,37.492532,127.118638
1,가락시장(340),하차,3206055,37.492532,127.118638
2,강남(222),승차,36264203,37.474136,127.093849
3,강남(222),하차,36727286,37.474136,127.093849
4,강변(214),승차,18308815,37.535111,127.094741
...,...,...,...,...,...
233,홍대입구(239),하차,29943564,37.556626,126.923514
234,홍제(314),승차,7504819,37.587100,126.946760
235,홍제(314),하차,7058774,37.587100,126.946760
236,회현(425),승차,10961805,37.557097,126.976229


In [475]:
sql = "INSERT INTO subway VALUES (%s, %s, %s, %s, %s)"
for idx, row in subway_df.iterrows():
    if idx == 0:
        continue
    cur.execute(sql,row.values.tolist())
    

In [476]:
con.commit()

In [472]:
con.commit()

In [ ]:

data = star_df.gugun_name.value_counts()
df = pd.DataFrame(data)
df.reset_index(inplace=True)
df.rename(columns={'count' : 'value'}, inplace=True)
folium.Choropleth(geo_str, 
                  data = df, 
                  columns=['gugun_name', 'value'],
                  fill_color= "PuRd",
                  key_on = 'feature.id').add_to(fmap)
for idx, row in star_df[['s_name','lat','lot']].iterrows():
    folium.Marker(row[['lat','lot']],popup=row.s_name).add_to(fmap)



In [499]:
DF = pd.DataFrame({'name':['김대건','김대건','김동연','김동연'],'hobby':['테니스','테니스','축구','테니스']})
DF[~DF['name'].duplicated()]

,name,hobby
0,김대건,테니스
2,김동연,축구


In [483]:
subway_df['구 분']

0      승차
1      하차
2      승차
3      하차
4      승차
       ..
233    하차
234    승차
235    하차
236    승차
237    하차
Name: 구 분, Length: 238, dtype: object

In [488]:
subway_df[subway_df[['역명','구 분']].duplicated()]

,역명,구 분,인원,위도,경도


In [502]:
geo_station

{'가락시장(340)': [37.4925324, 127.1186382],
 '강남(222)': [37.474135849999996, 127.09384934901931],
 '강변(214)': [37.5351109, 127.0947415],
 '건대입구(212)': [37.5400566, 127.0702592],
 '경복궁(317)': [37.57975395, 126.97668069886859],
 '경찰병원(341)': [37.495401, 127.1239038],
 '고속터미널(329)': [37.5119797, 127.0017181],
 '교대(223)': [37.4939376, 127.0145997],
 '교대(330)': [37.4936592, 127.0137025],
 '구로디지털단지(232)': [37.48417755, 126.89575893688135],
 '구의(213)': [37.5368976, 127.085103],
 '구파발(310)': [37.6370259, 126.91910713728154],
 '금호(324)': [37.5358735, 126.86699754955583],
 '길음(417)': [37.6024214, 127.024414],
 '낙성대(227)': [37.4770291, 126.9635445],
 '남태령(434)': [37.4646556, 126.9886975],
 '노원(411)': [37.640818499999995, 127.05802997538439],
 '녹번(313)': [37.6009733, 126.9357307],
 '당고개(409)': [37.6695753, 127.0784401],
 '당산(237)': [37.533531, 126.9026845],
 '대림(233)': [37.5481738, 126.87587403345886],
 '대청(337)': [37.49509995, 127.07710515142998],
 '대치(335)': [37.50167935, 127.05486860718736],
 '도곡(

In [504]:
import json
with open('./subway_station.json','w', encoding='utf-8') as f:
    json.dump(geo_station,f,indent=4,ensure_ascii=False)